In [1]:
import networkx as nx;
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def MST_MFNIP (G, budget, network, gamma):
    
    mst_file = open('MST_MFNIP_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    
    mst_file.close();

In [3]:
def MaxFlow (network, Gamma):
    
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
    maxFlow = gp.Model("MaxFlow"); 
    
    x = maxFlow.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    maxFlow.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) - 
                       gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
    
    for i,j in G.edges:
        maxFlow.addConstr(x[i,j] <= G.edges[i,j]['capacity']*(1-Gamma[i,j]));
            
    maxFlow.setObjective(x[t,s], GRB.MAXIMIZE);
    
    maxFlow.update();
    maxFlow.setParam("OutputFlag", 0);
    maxFlow.optimize();
    
    X = {};
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
    
    return X;

In [4]:
# Max Problem (to obtain the values of x and z given an interdiction)

def InnerProblem (network, Gamma, M, s, t):
    
    networkCSV = network+'.csv';
    
    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
  
    max_model = gp.Model("Max_z");
    
    x = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = max_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sum_z = max_model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = M);
            
    max_model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                         - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    for i,j in G.edges:
        max_model.addConstr(x[i,j] <= (1-Gamma[i,j])*G.edges[i,j]['capacity']);
            
    for i,j in G.edges:
        max_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
            
    max_model.addConstr(alpha[t] - alpha[s] >= 1);
            
    max_model.addConstr(x[t,s] >= gp.quicksum((1-Gamma[i,j])*G.edges[i,j]['capacity']*theta[i,j]
                                                     for i,j in G.edges));
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            max_model.addConstr(x[i,j] >= (1/M)*z[i,j]);
            
    max_model.addConstr(sum_z <= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
    max_model.setObjective(sum_z, GRB.MAXIMIZE); 
            
    max_model.setParam("IntegralityFocus",1);
    #max_model.setParam("NumericFocus",2);       
    #max_model.setParam('TimeLimit', T_Lim);
            
    max_model.update();
    max_model.setParam("OutputFlag", 0);
    max_model.optimize();
    
    if max_model.SolCount == 0:
        print("No feasible solution available (Max-Problem)");
    
    
    X = {};
    Z = {};
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Z[i,j] = z[i,j].x;
      
    return X, Z;

In [5]:
def solver_MFNIP (network, budget, T_Limit, summaryName):
    
    start_time = time.time(); 
    
    mod_network = 'Modified_'+network;
    
    networkCSV = network+'.csv';
    mod_networkCSV = 'Modified_'+network+'.csv';
    
    
    # Reading network file
    with open(mod_networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(mod_networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);
    
    A = 0;
    U = G.edges[t,s]['capacity'];
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
            
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###

    
    ### Optimization Model ###
    
    M = A;
    
    #instance = 'LogFile_'+MFNIP_network+'_b'+str(budget)+'.txt';
    
    model = gp.Model("MFNIP_method"); 
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY, name="gamma"); 
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="x", lb = 0, ub = GRB.INFINITY);
    theta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="theta", lb = 0, ub = 1); 
    alpha = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="alpha", lb = 0, ub = 1); 
    beta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="beta", lb = 0, ub = 1); 
    
    
    model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    model.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
    
    
    
    model.setObjective(x[t,s], GRB.MINIMIZE);
    
    model.setParam("IntegralityFocus",1);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("OutputFlag", 0);
    model.optimize();
  
    status = model.status;
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    LB = model.objBound;
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    
    # ~~~~~~~~~~~ MAX FLOW ON THE ORIGINAL NET ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    Gamma = {};
    for i,j in G.edges:
        Gamma[i,j] = gamma[i,j].x;
    
    X, Z = InnerProblem (network, Gamma, M, s,t);
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    # ~~~~~~~~~~ CREATING MST (MIP START) FILE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    #MST_MFNIP (G, budget, network, gamma);
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    
    
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('Results_'+mod_network+'_b'+str(budget)+'.txt', "w");
    file.write('Instance: %s, budget %g \n' %(mod_network, budget));
    file.write('Instance executed at: %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g \n' % (t+1));
    file.write('Number of Arcs: %g \n' % G.number_of_edges());
    file.write('Number of special arcs: %g\n' %A);
    file.write('Level of special set: %g \n' % calA_level);
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n')
    print('Instance: %s, budget %g\n' %(mod_network, budget));
     
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' %status);
        file.write('Status: %g \n' %status);
    
    print('Max-Flow (Original Network): %g \n' %X[t,s]);
    file.write('Max-Flow (Original Network): %g \n' %X[t,s]);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    file.write('Level 0 Capacities: %g \n\n ' %G.edges[s,1]['capacity']);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g" %(i,j, gamma[i,j].x));
    print('\n');
    
    
    lev1 = 0;
    lev2 = 0;
    lev3 = 0;
    other_level = 0;
    
    file.write('\nInterdiction plan: \n');
    
    key1 = False;
    key2 = False;
    key3 = False;
    
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001:
            if G.edges[i,j]["level"] == 1:
                if key1 == False:
                    file.write("Level 1:" +'\n');
                    key1 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev1 += 1;
            elif G.edges[i,j]["level"] == 2:
                if key2 == False:
                    file.write('Level 2: \n');
                    key2 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev2 += 1;
            elif G.edges[i,j]["level"] == 3:
                if key3 == False:
                    file.write('Level 3: \n');
                    key3 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev3 += 1;
            else:
                file.write('Other Levels: \n');
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                other_level += 1;
    '''
    file.write('\nSpecial arcs with flow : \n');    
    print('Special arcs with flow : '); 
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if x[i,j].x > 0.0001:
                file.write('Arc (%s,%s), Flow: %f \n' %(i,j, x[i,j].x));
                print('Arc (%s,%s), Flow: %f ' %(i,j, x[i,j].x));
    '''            
    
    file.write('\nSpecial arcs with flow (original net) : \n');    
    print('Special arcs with flow (original net): '); 
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if X[i,j] > 0.0001:
                file.write('Arc (%s,%s), Flow: %f \n' %(i,j, X[i,j]));
                print('Arc (%s,%s), Flow: %f ' %(i,j, X[i,j]));
        
                  
    print('\n');
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, calA_level, (t+1), G.number_of_edges(), A, OptGap, obj, X[t,s], lev1, lev2, lev3,
                 other_level, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();

In [6]:
n_Networks = 10;
Budget = [0,3,6,9,12];

T_Limit = 7200;


summaryName = "MFNIPmethod_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance,Budget,Cal_A,Nodes,Arcs,SpecialArcs,OptGap,');
file_summary.write('ObjVal,MaxFlow,Int_L1,Int_L2,Int_L3,Other,Time\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Net'+str(n);
    
    for budget in Budget:
        solver_MFNIP (network, budget, T_Limit, summaryName);

Set parameter TokenServer to value "license8.clemson.edu"
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instance: Modified_Net1, budget 0

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 16
Max-Flow (Original Network): 286 

run time: 0.07 sec 

Intediction:


Special arcs with flow (original net): 
Arc (1,6), Flow: 21.000000 
Arc (1,7), Flow: 10.937500 
Arc (1,8), Flow: 30.000000 
Arc (2,6), Flow: 16.000000 
Arc (2,7), Flow: 16.000000 
Arc (2,8), Flow: 16.937500 
Arc (2,9), Flow: 9.000000 
Arc (3,8), Flow: 21.000000 
Arc (3,9), Flow: 27.000000 
Arc (3,10), Flow: 25.000000 
Arc (4,6), Flow: 18.000000 
Arc (4,7), Flow: 0.062500 
Arc (4,9), Flow: 27.000000 
Arc (4,10), Flow: 0.062500 
Arc (5,8), Flow: 22.000000 
Arc (5,10), Flow: 26.000000 


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instan